## Regional Analysis

### Steps

1. Load regions data
2. Loop over regions. For each:
    - Calculate the variance in co2 flux seasonal amplitude
3. Rank the regions by amplitude
4. Select quantile of regions with highest variance

### Notes:

Koppen Geiger data from : http://koeppen-geiger.vu-wien.ac.at/


In [ ]:
import os
import pandas as pd
import rioxarray as rxr
import xarray as xr
import geopandas as gpd
import rasterio as ras
from matplotlib import pyplot
from rasterio.plot import show
import matplotlib as mpl
from matplotlib.colors import ListedColormap
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [ ]:
# reading in regions and setting nodata values

# koga = rxr.open_rasterio('../data_input/KoeppenGeiger3_KG3_CRUTS32_Hist_7100.tif', mask_and_scale=False)
koga = rxr.open_rasterio(
    '../data_input/VU-VIENA/KG_1986-2010.grd', mask_and_scale=False)
koga.rio.write_nodata(32, inplace=True)
koga = koga.where(koga != koga.rio.nodata)
koga.rio.write_nodata(32, encoded=True, inplace=True)
koga = koga.rio.reproject("EPSG:4326")
print(koga.rio.crs)
# print(f"nodata: {koga.rio.nodata}")
# print(f"encoded_nodata: {koga.rio.encoded_nodata}")


In [ ]:

# Plot regions to check all looks good.

# Color list and labels (including oceans as layer 32)
koga_cols = np.array(["#960000", "#FF0000", "#FF6E6E", "#FFCCCC", "#CC8D14", "#CCAA54", "#FFCC00", "#FFFF64", "#007800", "#005000",
          "#003200", "#96FF00", "#00D700", "#00AA00", "#BEBE00", "#8C8C00", "#5A5A00", "#550055", "#820082", "#C800C8",
          "#FF6EFF", "#646464", "#8C8C8C", "#BEBEBE", "#E6E6E6", "#6E28B4", "#B464FA", "#C89BFA", "#C8C8FF", "#6496FF",
          "#64FFFF", "#F5FFFF"])
koga_labels = np.array(['Af', 'Am', 'As', 'Aw', 'BSh', 'BSk', 'BWh', 'BWk', 'Cfa', 'Cfb',
            'Cfc', 'Csa', 'Csb', 'Csc', 'Cwa', 'Cwb', 'Cwc', 'Dfa', 'Dfb', 'Dfc',
            'Dfd', 'Dsa', 'Dsb', 'Dsc', 'Dsd', 'Dwa', 'Dwb', 'Dwc', 'Dwd', 'EF',
            'ET', 'Ocean'])
# If setting ocean layer 32 as missing, do:
koga_cols = koga_cols[0:-1]
koga_labels = koga_labels[0:-1]

regind = np.unique(koga.data[~np.isnan(koga.data)]).astype(int)


In [ ]:
# Plotting function: map plot of flux trends in the northern hemisphere

def plot_global(rast, cols, ticks, labels):
    cmap = ListedColormap(koga_cols)
    fig = plt.figure(figsize=[17, 6])
    ax = fig.add_subplot(
        111,
        projection=ccrs.PlateCarree(  # projection=ccrs.Orthographic(0, 90)
            central_longitude=0.0,),
        facecolor="white"
    )
    p = rast.plot(
        ax=ax,
        transform=ccrs.PlateCarree(),
        cmap=cmap,
        add_colorbar=False
    )
    # ax.set_global()
    ax.coastlines()
    ax.set_title('Koeppen-Geiger Regions')
    cbar = fig.colorbar(p, label=None, ticks=ticks)
    cbar = cbar.ax.set_yticklabels(labels)


In [ ]:

plot_global(koga, koga_cols, regind, koga_labels)

## Regional analysis

In [ ]:
# read in files: co2 data and continent vector file
conts = gpd.read_file('../data_input/continents.geojson')
co2fluxamp = rxr.open_rasterio('../data_output/co2invSeasAmp_s99.nc')


# Create a template dataframe to save results
regind = np.unique(koga.data[~np.isnan(koga.data)]).astype(int)
df_regions = pd.DataFrame({'regind': regind, 'regname': koga_labels,
                          'regcolor': koga_cols, 'area': np.nan, 'co2amp_var': np.nan})
# Create a dictionary to hold dataframes for each continent
conts_dict = dict()

# Create area grid for co2fluxamp grid cells
latcos = np.cos(np.deg2rad(co2fluxamp.y))
latcos.name = "weights"
area = co2fluxamp[1].squeeze()
# Aproximate grid area at Equator
area = area.where(area.isnull(), other=(2 * 111) * (2.5 * 111))


In [ ]:
# Function to analyse for a chosen continent

def reg_analysis(cont_name):
    # cont_name is the continent name string as found in the conts geopandas object
    cont = conts[conts['CONTINENT'] == cont_name]
    koga_sel = koga.rio.clip(cont.geometry, cont.crs, all_touched=True)
    koga_sel = koga_sel.squeeze()  # Get rid of the 'band' dimension
    del koga_sel['band']
    koga_sel = koga_sel.rio.reproject_match(co2fluxamp)
    
    # Make copy of DF to store results
    df_results = df_regions.copy()

    # Create list of region indices
    region_index = np.unique(koga_sel.data[~np.isnan(koga_sel.data)]).astype(int)

    plot_global(koga_sel, koga_cols[region_index-1], region_index, koga_labels[region_index-1])
    # plot_global(koga_sel, np.array(koga_cols)[region_index-1], region_index, np.array(koga_labels)[region_index-1])


    # Loop over region indices and calculate statistics of CO2 fluxes
    for i in range(len(region_index)):  # [15]: #
        roi = region_index[i]
        reg_co2amp = co2fluxamp.where(koga_sel == roi)
        reg_area = area.where(koga_sel == roi).weighted(latcos)
        totarea = reg_area.sum().data
        mean_var = reg_co2amp.var(dim='year').mean().data
        df_results.loc[df_results['regind'] ==
                       roi, 'ampvar'] = np.round(mean_var, 3)
        df_results.loc[df_results['regind'] == roi, 'area'] = np.round(totarea)

        # fig, ax = pyplot.subplots()
        # regfamp[1].plot(ax = ax)#, color = koga_cols[int(roi)-1])
        # conts.plot(ax = ax, facecolor="none", edgecolor="black")

    df_results['area_ampvar'] = np.round(
        df_results['area'] * df_results['ampvar'])
    df_results = df_results.sort_values('area_ampvar', ascending=False)
    df_results['area_ampvar_sum'] = df_results['area_ampvar'].cumsum()
    amparea_sum = df_results['area_ampvar'].sum()

    print('Total amparea sum = : ', amparea_sum, '\n')
    print('80% of amparea sum = : ', amparea_sum * 0.8, '\n')

    # Explore data
    df_results.sort_values('area', ascending=False).plot.bar('regname', 'area')
    df_results.sort_values('ampvar', ascending=False).plot.bar(
        'regname', 'ampvar')
    df_results.sort_values('area_ampvar', ascending=False).plot.bar(
        'regname', 'area_ampvar')

    return (df_results)


### Analysis for each NH continent

In [ ]:
df_co2_as = reg_analysis("Asia")

In [ ]:
df_co2_eu = reg_analysis("Europe")

In [ ]:
# Calculating total area

2633051.0